In [13]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   


# Load data

In [14]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, status, _ = fbg.eval_global_conn_k(states, G_base, target_k=2)
print("k =", k_val, "status =", status)

k = 2 status = s


In [15]:
s_fun = lambda comps_st: fbg.eval_global_conn_k(comps_st, G_base, target_k=2)
row_names = list(edges.keys()) + ['sys']
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names[:-1]]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [ ]:
TSUMPATH = Path("tsum_res") 

rules_mat_surv = torch.load(TSUMPATH / "rules_surv.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / "rules_fail.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

C:\Users\jb622s\AppData\Local\Temp\ipykernel_38732\3373689900.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rules_mat_surv = torch.load(r"toy_tsum/rules_surv.pt", map_

# Calculate system probabilities

## Marginal probability

In [17]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun
)
print(f"P(sys=1) = {pr_cond['survival']:.3f}")
print(f"P(sys=0 ) = {pr_cond['failure']:.3f}\n")

P(sys=1) = 0.168
P(sys=0 ) = 0.832



## Conditional probability given "one" component state

In [18]:
for x in row_names[:-1]:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun
    )
    print(f"P(sys=1 | {x}=1) = {pr_cond['survival']:.3f}")
    print(f"P(sys=0 | {x}=1) = {pr_cond['failure']:.3f}\n")

Eval P(sys | e01=1)
P(sys=1 | e01=1) = 0.168
P(sys=0 | e01=1) = 0.832

Eval P(sys | e02=1)
P(sys=1 | e02=1) = 0.210
P(sys=0 | e02=1) = 0.790

Eval P(sys | e03=1)
P(sys=1 | e03=1) = 0.210
P(sys=0 | e03=1) = 0.790

Eval P(sys | e04=1)
P(sys=1 | e04=1) = 0.209
P(sys=0 | e04=1) = 0.791

Eval P(sys | e05=1)
P(sys=1 | e05=1) = 0.209
P(sys=0 | e05=1) = 0.791

Eval P(sys | e06=1)
P(sys=1 | e06=1) = 0.168
P(sys=0 | e06=1) = 0.832

Eval P(sys | e07=1)
P(sys=1 | e07=1) = 0.210
P(sys=0 | e07=1) = 0.790

Eval P(sys | e08=1)
P(sys=1 | e08=1) = 0.210
P(sys=0 | e08=1) = 0.790

Eval P(sys | e09=1)
P(sys=1 | e09=1) = 0.210
P(sys=0 | e09=1) = 0.790

Eval P(sys | e10=1)
P(sys=1 | e10=1) = 0.168
P(sys=0 | e10=1) = 0.832

Eval P(sys | e11=1)
P(sys=1 | e11=1) = 0.211
P(sys=0 | e11=1) = 0.789



## Conditional probability given "two" component state

In [19]:
for i, x in enumerate(row_names[:-1]):
    for j, y in enumerate(row_names[i+1:-1]):
        
        print(f"Eval P(sys | {x}=1, {y}=1)")
        pr_cond = tsum.get_comp_cond_sys_prob(
            rules_mat_surv,
            rules_mat_fail,
            probs,
            comps_st_cond = {x: 1, y: 1},
            row_names = row_names,
            s_fun = s_fun
        )
        print(f"P(sys=1 | {x}=1, {y}=1) = {pr_cond['survival']:.3f}")
        print(f"P(sys=0 | {x}=1, {y}=1) = {pr_cond['failure']:.3f}\n")
   

Eval P(sys | e01=1, e02=1)
P(sys=1 | e01=1, e02=1) = 0.210
P(sys=0 | e01=1, e02=1) = 0.790

Eval P(sys | e01=1, e03=1)
P(sys=1 | e01=1, e03=1) = 0.209
P(sys=0 | e01=1, e03=1) = 0.791

Eval P(sys | e01=1, e04=1)
P(sys=1 | e01=1, e04=1) = 0.209
P(sys=0 | e01=1, e04=1) = 0.791

Eval P(sys | e01=1, e05=1)
P(sys=1 | e01=1, e05=1) = 0.210
P(sys=0 | e01=1, e05=1) = 0.790

Eval P(sys | e01=1, e06=1)
P(sys=1 | e01=1, e06=1) = 0.168
P(sys=0 | e01=1, e06=1) = 0.832

Eval P(sys | e01=1, e07=1)
P(sys=1 | e01=1, e07=1) = 0.210
P(sys=0 | e01=1, e07=1) = 0.790

Eval P(sys | e01=1, e08=1)
P(sys=1 | e01=1, e08=1) = 0.210
P(sys=0 | e01=1, e08=1) = 0.790

Eval P(sys | e01=1, e09=1)
P(sys=1 | e01=1, e09=1) = 0.209
P(sys=0 | e01=1, e09=1) = 0.791

Eval P(sys | e01=1, e10=1)
P(sys=1 | e01=1, e10=1) = 0.168
P(sys=0 | e01=1, e10=1) = 0.832

Eval P(sys | e01=1, e11=1)
P(sys=1 | e01=1, e11=1) = 0.210
P(sys=0 | e01=1, e11=1) = 0.790

Eval P(sys | e02=1, e03=1)
P(sys=1 | e02=1, e03=1) = 0.262
P(sys=0 | e02=1, e03=